In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

def get_grad_norm(model):
    grad_all = 0.0
    for p in model.parameters():
        grad = 0.0
        if p.grad is not None:
            grad = (p.grad.cpu().data.numpy() ** 2).sum()
        grad_all += grad
    return grad_all ** 0.5

# helper function for updating model based on loss of given x inputs and correct y outputs
# returns loss from criterion
def update_model(x_i, y_i, model, optimizer, criterion, no_unsqueeze=False, no_float=False, grad_norm=False, device=None):
    if not no_float:
        output = model(x_i.float())
    else:
        output = model(x_i)

    if not no_unsqueeze:
        y_i = y_i.unsqueeze(1)

    if not no_float:
        loss = criterion(output, y_i.float())
    else:
        loss = criterion(output, y_i)

    if grad_norm:
        loss = criterion(get_grad_norm(model), 0)


    #print(loss)
    out_loss = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    return out_loss

# Neural network with linear hidden layers
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_layers, num_classes, hidden_size):
        assert hidden_size >= 1
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        # input layer
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size, bias=False)])
        # hidden layers
        for i in range(num_layers):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
            self.layers.append(nn.ReLU())
        # output layer
        self.layers.append(nn.Linear(hidden_size, num_classes, bias=False))
        #print(self.layers)

    def forward(self, x):
        out = x
        for i, layer in enumerate(self.layers):
            out = layer(out)
            #print(i, out.size())
        return out

    def num_parameters(self):
        return self.input_size*self.hidden_size + self.num_layers*(self.hidden_size*self.hidden_size) + self.hidden_size*self.num_classes


In [4]:
import numpy as np
from torch.utils.data import Dataset
import math

class FunctionDataset(Dataset):
    def __init__(self):
        self.x = np.linspace(-5,5,2000)
        x_term = 5*math.pi*self.x
        self.y = np.array(list(map(math.sin,x_term)))/x_term
        self.x_y = [(x,y) for x, y in zip(self.x, self.y)]

    def __len__(self):
        return len(self.x_y)

    def __getitem__(self, index):
        _x = self.x_y[index][0]
        _y = self.x_y[index][1]

        return _x, _y

class TimeSeriesDataset(Dataset):
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        _x = self.X[index]
        _y = self.Y[index]
        return _x, _y

In [6]:
import matplotlib.pyplot as plt
import random
import pylab as pl
import numpy as np
from torch.utils.data import Dataset
import math

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


#
# hyper parameters
# 
input_size = 1
output_size = 1
batch_size = 100
learning_rate = 0.001
num_epochs = 10
training_times = 1
function_dataset = FunctionDataset()


# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.MSELoss()

xy_train = [x for x in function_dataset.x_y]
#print(len(xy_train))
training_set = TimeSeriesDataset([x[0] for x in xy_train], [y[1] for y in xy_train])

#xy_test = [x for x in function_dataset.x_y[int(.9*len(function_dataset.x_y)):]]
#print(len(xy_test))
#testing_set = TimeSeriesDataset([x[0] for x in xy_test], [y[1] for y in xy_test])

# Data loaders
train_loader = torch.utils.data.DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)

#model
num_layers = 4
hidden_size = 22
model = NeuralNet(input_size, num_layers=num_layers, hidden_size=hidden_size, num_classes=output_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print("model.num_parameters()",model.num_parameters(),num_layers,hidden_size)

# Training
n_total_steps = len(train_loader)
mean_epoch_loss = []
batch_loss = []
grad_norm_list = []
for epoch in range(num_epochs):
    loss_list = []
    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.reshape(-1, input_size).to(device)
        y_i = y_i.to(device)
        if epoch >= num_epochs/2:
            loss_list.append(update_model(x_i, y_i, model, optimizer, criterion, grad_norm=True,device=device))
        else:
            loss_list.append(update_model(x_i, y_i, model, optimizer, criterion))
        grad_norm_list.append(get_grad_norm(model))

        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss_list[-1]:.4f}')

    batch_loss.extend(loss_list)
    mean_epoch_loss.append(sum(loss_list)/len(loss_list))
    print()

fig, ax = plt.subplots(3,1)
fig.tight_layout()
fig.set_size_inches(14,13.5)
ax[0].plot(range(len(batch_loss)), batch_loss,label='model loss',color='green')
ax[0].set_yscale('log')
ax[0].set_title('Model Loss')
ax[0].set_xlabel('Iteration')
ax[0].set_ylabel('Loss')
ax[0].legend()

with torch.no_grad():
    X = torch.from_numpy(np.array(training_set.X).astype(np.float32)).to(device)
    X = X.reshape(len(X),1)
    y_predicted = model(X)

print(grad_norm_list)
ax[1].plot(range(len(grad_norm_list)), grad_norm_list,label='model grad norm',color='green')
ax[1].set_xlabel('Iteration')
ax[1].set_ylabel('Grad Norm')

ax[2].plot(function_dataset.x, function_dataset.y, color='black',label=r"$\frac{\mathrm{sin}(5 \pi x)}{5 \pi x}$")
ax[2].xaxis.set_ticks_position('bottom')
ax[2].yaxis.set_ticks_position('left')
ax[2].set_xlim([0.01,1])
ax[2].set_ylim([-.25,1])
print(y_predicted.cpu())
ax[2].plot(training_set.X, y_predicted.cpu(), color='green', label='model_2')
ax[2].legend()

model.num_parameters() 1980 4 22
Epoch [1/10], Step [10/20], Loss: 0.0282
Epoch [1/10], Step [20/20], Loss: 0.0171

Epoch [2/10], Step [10/20], Loss: 0.0134
Epoch [2/10], Step [20/20], Loss: 0.0066

Epoch [3/10], Step [10/20], Loss: 0.0150
Epoch [3/10], Step [20/20], Loss: 0.0146

Epoch [4/10], Step [10/20], Loss: 0.0071
Epoch [4/10], Step [20/20], Loss: 0.0176

Epoch [5/10], Step [10/20], Loss: 0.0180
Epoch [5/10], Step [20/20], Loss: 0.0145



AttributeError: 'int' object has no attribute 'size'